In [2]:
##
## Import libraries
import pandas as pd
import numpy as np
from sklearn import linear_model, model_selection, preprocessing, metrics, svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# True Positive [TP] = your prediction is 1, and the ground truth is also 1 - you predicted a positive and that's true!
# False Positive [FP] = your prediction is 1, and the ground truth is 0 - you predicted a positive, and that's false.
# False Negative [FN] = your prediction is 0, and the ground truth is 1 - you predicted a negative, and that's false.

##
## Accuracy is measured by F1 score = 2 ∗ (precision∗recall) / (precision+recall)
## 
## and precision = TP/(TP+FP) and recall = TP/(TP+FN)

### Read Data

In [3]:
# read data
rootPath = '/Users/joaquimlyrio/Documents/Kaggle/NLP with Disaster Tweets/'
train = pd.read_csv( rootPath + 'data/nlp-getting-started/train.csv' )
test  = pd.read_csv( rootPath + 'data/nlp-getting-started/test.csv' )
subm_samp = pd.read_csv( rootPath + 'data/nlp-getting-started/sample_submission.csv' )

In [4]:
train.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [5]:
##
## Checking size of each class on train set
print( train[ train['target'] == 1 ].shape[0] )
print( train[ train['target'] == 0 ].shape[0] )

3271
4342


In [6]:
# check first tweets when disaster happened
train[train["target"] == 1]["text"].values[0:2]

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada'], dtype=object)

In [8]:
# check first tweets when disaster NOT happened
train[train["target"] == 0]["text"].values[0:2]

array(["What's up man?", 'I love fruits'], dtype=object)

## Feature Engineering

### Word Counts

In [9]:

##
## Count words
count_vect = CountVectorizer()
train_counts = count_vect.fit_transform(train.text)
train_counts.shape

(7613, 21637)

### TF-IDF

In [10]:

##
## TF-IDF
tfidf_transformer = TfidfTransformer()
train_tfidf = tfidf_transformer.fit_transform(train_counts)
train_tfidf.shape

(7613, 21637)

## Models

### Bernoulli Naive Bayes

In [11]:
##
## Fit Bernoulli Naive-Bayes
clf = BernoulliNB().fit(train_tfidf, train.target)
clf

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [12]:

##
## Predict on train set
nObs = 100000
print( np.mean( clf.predict(train_tfidf[0:nObs]) == train.target[0:nObs] ) )
metrics.f1_score( y_true = clf.predict(train_tfidf[0:nObs]),
                  y_pred = train.target[0:nObs] )

0.8943911729935636


0.8658658658658658

In [13]:
## Bernoulli NB
clf1 = BernoulliNB()
scores1 = cross_val_score(clf1, train_tfidf, train.target, cv=5, scoring='f1')
f'BernoulliNB: {scores1}'

'BernoulliNB: [0.63339383 0.62758051 0.68714632 0.64700781 0.76566125]'

In [61]:
# ## SVM
# clf2 = svm.SVC(kernel='linear', C=1)
# scores2 = cross_val_score(clf2, X_train_tfidf, train.target, cv=5, scoring='f1')
# f'SVM: {scores2}'

'SVM: [0.62643678 0.55838455 0.62658764 0.60056127 0.73311897]'

In [63]:
# ## RandomForestClassifier
# clf3 = RandomForestClassifier()
# scores3 = cross_val_score(clf3, X_train_tfidf, train.target, cv=5, scoring='f1')
# f'RandomForestClassifier: {scores3}'

'SVM: [0.53548387 0.48820513 0.52991453 0.5257732  0.68392857]'

## Ridge Classifier

In [25]:
##
## Obs: this cell takes a while to run ~ 3 min or so
##

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import RidgeClassifier

## RidgeClassifier
ridge = linear_model.RidgeClassifier()
ridge.get_params()

# define parameter range to perform GridSearchCV
param_grid = [ {'alpha': np.arange(0,1.1,.1)} ]

# Split the dataset for cross validation
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(train_tfidf, 
                                                          train.target, 
                                                          test_size=0.2, 
                                                          random_state=0)


# Set the parameters by cross-validation
tuned_parameter = [{'alpha':[0,.25,.5,.75,1.0]}]

# Score to to optimize over
score = 'f1'

clf = GridSearchCV(
    RidgeClassifier(), tuned_parameter, scoring=score
)

clf.fit(X_train_tfidf, y_train)


GridSearchCV(cv=None, error_score=nan,
             estimator=RidgeClassifier(alpha=1.0, class_weight=None,
                                       copy_X=True, fit_intercept=True,
                                       max_iter=None, normalize=False,
                                       random_state=None, solver='auto',
                                       tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid=[{'alpha': [0, 0.25, 0.5, 0.75, 1.0]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [27]:
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(X_test_tfidf)
print(classification_report(y_true, y_pred))
print()

Best parameters set found on development set:

{'alpha': 0.75}

Grid scores on development set:

0.409 (+/-0.161) for {'alpha': 0}
0.737 (+/-0.018) for {'alpha': 0.25}
0.745 (+/-0.025) for {'alpha': 0.5}
0.751 (+/-0.032) for {'alpha': 0.75}
0.750 (+/-0.032) for {'alpha': 1.0}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.80      0.90      0.85       886
           1       0.83      0.69      0.75       637

    accuracy                           0.81      1523
   macro avg       0.81      0.79      0.80      1523
weighted avg       0.81      0.81      0.81      1523




In [ ]:
###
### Try to do word embeddings + classification model
###
